<a href="https://colab.research.google.com/github/pascalfriederich/programming_exercises/blob/main/exercise04_lock_in_amp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np